# OPER 618 Game Theory and Math Programming
## Homework #3 - Computing Concepts for Normal Form Games
## Hosley, Brandon

In [1]:
import pyomo.environ as pyo

1. **Maxmin Strategies**. Consider the two-player normal form game depicted in Figure 4.1 of
the textbook (and shown below), as well as the file *"OPER 618 - Homework 3 - Problem 1 -
Sets and Parameters.py"* posted on Canvas.

<div style="margin-left: auto;
            margin-right: auto;
            width: 30%">

|              |       | Player 2 |       |
|:------------:|:-----:|:--------:|:-----:|
|              |       |   $k_1$  | $k_2$ |
|              | $j_1$ |   0, 1   |  6, 0 |
| **Player 1** | $j_2$ |   2, 0   |  5, 2 |
|              | $j_3$ |   3, 4   |  3, 3 |

</div>

In [2]:
%reset -f
m = pyo.ConcreteModel()

# Input sets
m.J = {1,2,3}
m.K = {1,2}

# Input parameter data
u1=[[0,0,0],
   [0, 0, 6], 
   [0, 2, 5],
   [0, 3, 3]]

u2=[[0,0,0],
   [0, 1, 0], 
   [0, 0, 2],
   [0, 4, 3]]

# Decision Variables
m.U1 = pyo.Var(domain=pyo.NonNegativeReals) # Player 1's utility
m.U2 = pyo.Var(domain=pyo.NonNegativeReals) # Player 2's utility
m.p = pyo.Var(m.J,m.K,domain=pyo.NonNegativeReals) # p[j,k]=p1[j]*p2[k]

<ol type="a">
  <li>
    For this non-zero sum game, use Python/Pyomo to formulate and solve the problem of
    finding the <i>maxmin</i> strategy for <b>Player 1</b>, allowing for mixed strategies. Discuss the
    significance of the optimal solution
  </li>
</ol>


<ol type="a" start="2">
  <li>
    For this non-zero sum game, use Python/Pyomo to formulate and solve the problem of
    finding the <i>maxmin</i> strategy for <b>Player 2</b>, allowing for mixed strategies. Discuss the
    significance of the optimal solution.
  </li>
</ol>

2. **Discriminating among Nash Equilibria with the LCP formulation**. Consider the two-player
normal form game depicted in Figure 4.1 of the textbook (and shown below), as well as the
file *"OPER 618 - Homework 3 - Problem 2 - LCP System of Equations.py"* posted on Canvas.

<div style="margin-left: auto;
            margin-right: auto;
            width: 30%">

|              |       | Player 2 |       |
|:------------:|:-----:|:--------:|:-----:|
|              |       |   $k_1$  | $k_2$ |
|              | $j_1$ |   0, 1   |  6, 0 |
| **Player 1** | $j_2$ |   2, 0   |  5, 2 |
|              | $j_3$ |   3, 4   |  3, 3 |

</div>

As formulated, the Python/Pyomo Concrete Model finds the Nash equilibrium $s = (s_1, s_2) = \left((0.67,0.33,0), (0.33, 0.67)\right)$ 
with expected payoffs of $(u_1, u_2) = (4, 0.67)$.
However, this is only one of the three Nash equilibria, and it doesn't seem to be a very
desirable one... at least not for Player 2.

In [ ]:
%reset -f
m = pyo.ConcreteModel()

# Input sets
m.J = {1,2,3}
m.K = {1,2}

# Input parameter data
u1=[[0,0,0],
   [0, 0, 6], 
   [0, 2, 5],
   [0, 3, 3]]

u2=[[0,0,0],
   [0, 1, 0], 
   [0, 0, 2],
   [0, 4, 3]]

# Decision Variables
m.U1 = pyo.Var(domain=pyo.NonNegativeReals) # Player 1's utility
m.U2 = pyo.Var(domain=pyo.NonNegativeReals) # Player 2's utility
m.p = pyo.Var(m.J,m.K,domain=pyo.NonNegativeReals) # p[j,k]=p1[j]*p2[k]

<ol type="a">
  <li>
    Modify and solve the Python/Pyomo formulation to find the Nash equilibrium that
    <u>minimizes the maximum difference in the players' utilities</u>.
  </li>
</ol>

<ol type="a" start="2">
  <li>
    Modify and solve the Python/Pyomo formulation with an objective function that will
    find the remaining Nash equilibrium. Discuss the merits of your approach as it pertains
    to generalizability for other two-player games.
  </li>
</ol>

<ol type="a" start="3">
  <li>
    Discuss method one could implement to find <u>all</u> Nash equilibria using the LCP formulation.
  </li>
</ol>

### Normal form game used for Problems 3, 4, & 5. 
Consider the two-player normal form game depicted below, which is a modified form of "Chicken".

<div style="margin-left: auto;
            margin-right: auto;
            width: 30%">

|              |       | Player 2 |       |
|:------------:|:-----:|:--------:|:-----:|
|              |       |   $k_1$  | $k_2$ |
| **Player 1** | $j_1$ |   0, 0   |  9, 3 |
|              | $j_2$ |   3, 9   |  7, 7 |

</div>
The following figure (Duffy and Feltovich, 2010) shows the possible payoffs for Player 1 and
Player 2, wherein the respective regions of feasible payoffs, correlated equilibrium payoffs,
and Nash equilibrium payoffs are depicted with the indicated letter(s).

![](https://www.mathworks.com/matlabcentral/mlc-downloads/downloads/e568cfd1-4a80-11e4-9553-005056977bd0/66a2877e-5aac-4a1d-b090-5035e35db9b2/images/screenshot.gif)

3. **Correlated Equilibria and Nash Equilibria**. Consider the Python/Pyomo models in the following files, which correspond to the two functional representations in Section 4.6.
    - *OPER 618 - Homework 3 - Problem 3 - Correlated Equilibrium (LP) - Chicken.py*
    - *OPER 618 - Homework 3 - Problem 3 - Nash Equilibrium (NLP) - Chicken.py*

    Modify the objective function(s) and resolve the model(s) as necessary to identify (a) the
    four correlated equilibria corresponding to the extreme points of the region of correlated
    equilibria payoff pairs from the diagram and (b) the three correlated equilibria
    corresponding to the three Nash equilibria corresponding to the extreme points of the
    region of NE payoff pairs from the diagram.

4. **The Region of Correlated Equilibria**. Identify a correlated equilibrium yielding a pair of
payoffs in the region $B_l$ (i.e., the region of correlated equilibria that are not Nash equilibria
and have lower-than-NE payoffs), as well as a correlated equilibrium yielding a pair of
payoffs in the region $B_h$.

5. **Unstable Solutions**. Show that the solution corresponding to the joint payoffs of $(7,7)$ is
neither a correlated equilibrium nor a Nash equilibrium.